## Precarga de librerias y funciones

In [6]:
%%capture
%run "../../recurrentes.ipynb"

In [7]:
%%capture
%run "../../funciones.ipynb"

#### Excel

In [8]:
# Activar Excel de seguimiento de medidas? 

excel = True
if excel == True:
    # Ingresar path excel para anotar resultados
    path_excel = base_path_l + r'\resultados_backtesting.xlsx'
    date = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
    

In [9]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) competencia_03_ct_sin_psi.parquet

dataset = dataset_clase_ternaria_l
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


## Formateo pre modelo

In [10]:

# [202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106] modelo 10 meses

ganancia_acierto = 273000
costo_estimulo = 7000

#Defenir mes de train y test

mes_train = 202106
mes_test = 202108

In [11]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)
    
# Binarización de la variable target

data = df_train
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

# Train-Test Split

if isinstance(mes_train, list):
    df_train = data[data['foto_mes'].isin(mes_train)]
else: df_test = data[data['foto_mes']==mes_test]

# definiendo X e Y

clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']

In [ ]:
predecir_punto_de_corte(  
    dataset,  
    mes_de_entrenamiento,  
    mes_de_prueba,  
    best_params,  
    best_iteration,  
    rango_cortes,  
    semillas,  
) 

In [ ]:
best_params = {

In [ ]:
predecir_punto_de_corte (df_train, 202106, 202108, )

## Back-testing

In [15]:
respuesta = ''
if respuesta == '':
   input (" 'personalizado' o 'default': ")
   if respuesta == 'personalizado':
      
      best_params = {'num_leaves': '',
      'learning_rate': '',
      'min_data_in_leaf': '',
      'feature_fraction': '',
      'bagging_fraction': ''}
   elif respuesta == 'default':
      params = {}


In [16]:
best_iter = 1789

df_voting = pd.DataFrame()

if respuesta == 'personalizado':
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': best_params['num_leaves'],
        'learning_rate': best_params['learning_rate'],
        'min_data_in_leaf': best_params['min_data_in_leaf'],
        'feature_fraction': best_params['feature_fraction'],
        'bagging_fraction': best_params['bagging_fraction'],
        'seed': x,
        'verbose': 0
    }

elif respuesta == 'default':
    params = {}


for x in semillas:
                
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)

    y_pred = model.predict(X_test)
    df_voting[f'prediccion_seed_{x}'] = y_pred

df_voting['prediccion'] = df_voting.mean(axis=1)
df_voting.index = X_test.index


[LightGBM] [Info] Number of positive: 966676, number of negative: 3768917
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.497922 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22084
[LightGBM] [Info] Number of data points in the train set: 4735593, number of used features: 154
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.204132 -> initscore=-1.360665
[LightGBM] [Info] Start training from score -1.360665
[LightGBM] [Info] Number of positive: 966676, number of negative: 3768917
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.422076 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22084
[LightGBM] [Info] Number of data points in the train set: 4735593, number of used features: 154
[Lig

In [17]:
prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'probabilidad': df_voting ['prediccion']}, index=X_test.index) 
prediccion.shape 

(165152, 2)

In [18]:
prediccion.isna().sum()

numero_de_cliente    0
probabilidad         0
dtype: int64

In [19]:
bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)
bajas.shape

(165152, 2)

In [20]:
bajas.isna().sum()

numero_de_cliente    0
clase_ternaria       0
dtype: int64

In [22]:
prediccion.shape
merged_data = prediccion.merge(bajas, on='numero_de_cliente', how='inner')

merged_data.columns
merged_data_sorted = merged_data.sort_values('probabilidad', ascending=False)


# Ordenar por probabilidad de mayor a menor

# Seleccionar top clientes
top_clients = merged_data_sorted.iloc[:12000].copy()  # Trabajar con una copia

# Convertir 'clase_ternaria' a variable binaria para los top clientes
top_clients.loc[:, 'bajas_reales'] = (top_clients['clase_ternaria'] == 'BAJA+2').astype(int)

top_clients.value_counts('clase_ternaria')


clase_ternaria
BAJA+2    11915
BAJA+1       85
Name: count, dtype: int64

In [24]:
top_clients.value_counts('bajas_reales')

bajas_reales
1    11915
0       85
Name: count, dtype: int64

In [15]:
dicc_medidas = calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test)

[LightGBM] [Info] Number of positive: 333766, number of negative: 970448
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.219239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 49926
[LightGBM] [Info] Number of data points in the train set: 1304214, number of used features: 295
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.255916 -> initscore=-1.067303
[LightGBM] [Info] Start training from score -1.067303
[LightGBM] [Info] Number of positive: 333766, number of negative: 970448
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.803860 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49926
[LightGBM] [Info] Number of data points in the train set: 1304214, number of used features: 295
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.255916 -> initscore=

In [16]:
if excel == True:
    dicc_medidas['fecha'] = date
    dicc_medidas['dataset'] = dataset_name
    

In [27]:
df_test['clase_ternaria'].value_counts()

clase_ternaria
BAJA+2    164077
BAJA+1      1075
Name: count, dtype: int64

In [29]:
df_train['clase_ternaria'].value_counts()

clase_ternaria
CONTINUA    3768917
BAJA+1       486980
BAJA+2       479696
Name: count, dtype: int64

In [26]:
bajas['clase_ternaria'].value_counts()

clase_ternaria
BAJA+2    164077
BAJA+1      1075
Name: count, dtype: int64

In [17]:
cantidad_columnas = df_train.shape[1]
mes_train = df_train['foto_mes'].max()
mes_test = df_test['foto_mes'].max()
# Agregar nota
consideraciones = 'entrenando sólo noviembre'

dicc_medidas['cantidad_columnas'] = cantidad_columnas
dicc_medidas['mes_train'] = mes_train
dicc_medidas['mes_test'] = mes_test
dicc_medidas['consideraciones'] = consideraciones

In [18]:
dicc_medidas 

{'accuracy_score': 0.9144144690555327,
 'precision_score': 0.10829671749806151,
 'recall_score': 0.8434826371414192,
 'f1_score': 0.19194869151921207,
 'roc_auc_score': 0.8793811837549762,
 'ganancia': 142786000,
 'fecha': '24-11-2024 17-17-52',
 'dataset': 'dataset_10_meses.parquet',
 'cantidad_columnas': 298,
 'mes_train': 202106,
 'mes_test': 202106,
 'consideraciones': 'entrenando sólo noviembre'}

In [20]:
if excel == True:
    anotar_excel(path_excel, dicc_medidas)


Primera celda vacía en la fila 21
Fila a actualizar: 21
se actualizó fila 21 en el excel
